In [ ]:
!pip install openpyxl

In [ ]:
!pip install EvalML

In [ ]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import evalml
from evalml.automl import AutoMLSearch
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Load the data

In [ ]:
base = '/kaggle/input/ipl-data-set/'
names = ['matches.csv','teamwise_home_and_away.csv','deliveries.csv','most_runs_average_strikerate.csv','teams.csv']
for i,name in enumerate(names):
    my_code = name[:-4]+'='+'pd.read_csv("'+base+name+'")'
    exec(my_code)

players = pd.read_excel(base+'Players.xlsx')

## Let's use AutoML to design initial base model fast

In [ ]:
matches['date'] = pd.to_datetime(matches['date'])
matches = matches.set_index('date')
matches.head()

## A lot of the features here are info about after the match which are to be discarded to avoid data leackage

In [ ]:
matches

In [ ]:
matches_auto = matches.copy()
matches_auto['winner_binary'] = matches_auto.apply(lambda x:1 if x[3] == x[9]else 0,axis=1)
matches_auto = matches_auto.drop('winner',axis=1)
matches_auto = matches_auto.drop(['id','umpire1','umpire2','umpire3','toss_winner','toss_decision','result','dl_applied','win_by_runs','win_by_wickets','player_of_match','venue'],axis=1)
matches_auto = matches_auto.astype("category")

## We will predict for the 2019 season

In [ ]:
X = matches_auto.drop('winner_binary',axis=1)
y = matches_auto['winner_binary']
X_train = X[X.index<='2018-12-30']
X_test = X[X.index>'2018-12-30']
y_train = y[y.index<='2018-12-30']
y_test = y[y.index>'2018-12-30']
# X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(X, y, problem_type='time series binary')

In [ ]:
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='binary')
automl.search()

> The best 3 models are very close and none is making a good job in the problem (due to low information of course, but we will compare our models to this base model

In [ ]:
log_loss(true_binary,list(exp.values))

## Let's try another base model which will guess the winner based on who won more matches in the past 

In [ ]:
matches_train = matches[matches.index<='2018-12-30']
matches_test = matches[matches.index>'2018-12-30']
w = matches_train.groupby('winner').count()['team1']
true_binary = []
expexted_binary = []
for i,row in matches_test.iterrows():
    t1_wins = 0
    t2_wins = 0
    t1 = row['team1']
    t2 = row['team2']
    winner = row['winner']
    if t1 in w:
        t1_wins = w[t1]
    if t2 in w:    
        t2_wins = w[t2]
    expected_winner = 0 if t1_wins > t2_wins else 1
    true_winner = 0 if t1==winner  else 1
    expexted_binary.append(expected_winner)
    true_binary.append(true_winner )

        


In [ ]:
from sklearn.metrics import log_loss
best_pipeline = automl.best_pipeline
exp = best_pipeline.predict(X_test)
log_loss(exp,true_binary)/len(exp)

In [ ]:
log_loss(expexted_binary,true_binary)/len(true_binary)

> by taking the mean log of the hard decision for both models there is no significant difference between them, so we can use any as a base line